In [3]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO

In [4]:
from PIL import Image, ImageDraw

In [5]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person

## Authenticate the client

In [7]:
#  creating a CognitiveServices credentials object with key and endpoint to create FaceClient Object
# Creating an authenticated FaceClient

face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

## Detect faces in an image

In [11]:
# Detect a face in a image that contains a single face
single_face_image_url = 'https://www.biography.com/.image/t_share/MTQ1MzAyNzYzOTgxNTE0NTEz/john-f-kennedy---mini-biography.jpg'
single_image_name = os.path.basename(single_face_image_url)

# we use detection model 2 beacuse we are not retriving attributes
detected_faces = face_client.face.detect_with_url(url=single_face_image_url, detectionModel='detection_02')

if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))
    
# Display the detected face ID in the first single-face image.
# Face ID's are used for comparision to faces (their IDs) detected in other images

for face in detected_faces: print(face.face_id)
print()

#saving this id for use in Find Similar
first_image_face_ID = detected_faces[0].face_id

4087f38a-1f7d-454e-b230-c3a2c7f113cd



In [14]:
# Lets try with the image which has no face
no_face_image_url = 'https://cdn.pixabay.com/photo/2015/12/01/20/28/road-1072823__340.jpg'
no_image_name = os.path.basename(no_face_image_url)

detected_faces = face_client.face.detect_with_url(url=no_face_image_url, detectionModel='detection_02')

if not detected_faces:
    raise Exception('No face detected from image {}'.format(no_image_name))
    


Exception: No face detected from image road-1072823__340.jpg

## Display and Frame Faces

In [19]:
single_face_image_url = 'https://raw.githubusercontent.com/Microsoft/Cognitive-Face-Windows/master/Data/detection1.jpg'
single_image_name = os.path.basename(single_face_image_url)
detected_faces = face_client.face.detect_with_url(url=single_face_image_url, detectionModel='detection_02')
if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))
    
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    
    return ((left,top),(right,bottom))

# Download the image from the url
response = requests.get(single_face_image_url)
img = Image.open(BytesIO(response.content))

# For each face returned use the face rectangle and draw a red box
print('Drawing rectangle aroung face... see popup for results.')
draw = ImageDraw.Draw(img)
for face in detected_faces:
    draw.rectangle(getRectangle(face), outline='red')
    
img.show()

Drawing rectangle aroung face... see popup for results.


## Find Matches

In [22]:
# run the code in detect faces in a image to save a reference to single face
# the following code to get references to several faces in a group image
# Detect the faces in a image that contains multiple faces; each detected face gets assigned a new ID

multi_face_image_url = 'http://www.historyplace.com/kennedy/president-family-portrait-closeup.jpg'
multi_image_name = os.path.basename(multi_face_image_url)
detected_faces2 = face_client.face.detect_with_url(url=multi_face_image_url,detectionModel='detection_02')

In [24]:
# finding the instances of first face in the group
# create list of the face IDs found in the second image
second_image_face_IDs = list(map(lambda x:x.face_id, detected_faces2))
# finding similar face IDs to first Id
similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
if not similar_faces[0]:
    print('No similar faces found in',multi_image_name,'.')

In [25]:
## Print Matches
print('Similar faces found in', multi_image_name + ':')
for face in similar_faces:
    first_image_face_ID = face.face_id
    face_info = next(x for x in detected_faces2 if x.face_id==first_image_face_ID)
    if face_info:
        print(' Face ID: ',first_image_face_ID)
        print(' Face rectangle:')
        print(' Left: ', str(face_info.face_rectangle.left))
        print(' Top: ', str(face_info.face_rectangle.top))
        print(' Width: ',str(face_info.face_rectangle.width))
        print(' Height: ',str(face_info.face_rectangle.height))

Similar faces found in president-family-portrait-closeup.jpg:
 Face ID:  5b3870e6-0d9e-4b65-a804-58684d51813e
 Face rectangle:
 Left:  413
 Top:  57
 Width:  55
 Height:  55


## Create and train Person group

**Create Person Group**

In [31]:
# SOURCE_PERSON_GROUP_ID should be all lowercase and alphanumeric
PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

# Used for the Delete Person Group example.
TARGET_PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

In [32]:
##creating person group and three person objects

# Create empty person group
print('Person group:', PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

# Define women friend
women = face_client.person_group_person.create(PERSON_GROUP_ID, 'Woman')
# Define Man friend
man = face_client.person_group_person.create(PERSON_GROUP_ID, "Man")
# Define child Friend
child = face_client.person_group_person.create(PERSON_GROUP_ID,'child')

Person group: e6fcc27f-7910-4fe5-857c-b6ccbbf0cf44


In [35]:
## Assign faces to persons

woman_images = [file for file in glob.glob('*.jpg') if file.startswith('w')]
man_images = [file for file in glob.glob('*.jpg') if file.startswith('m')]
child_images = [file for file in glob.glob('*.jpg') if file.startswith('ch')]

# Add to women person
for image in woman_images:
    w = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, women.person_id, w)
    
# Add to man person
for image in man_images:
    m = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, man.person_id, m)
    
# ADD to child person
for image in child_images:
    ch = open(image,'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, child.person_id, ch)

In [36]:
## Train person group

print()
print('Training Person group')
#Training the person group
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print('Training status: {}'.format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        sys.exit('Training the person group has failed.')
    time.sleep(5)


Training Person group
Training status: succeeded



## Identify a Face

In [37]:
# to identify a image among a set of images

# getting a test image
test_image_array = glob.glob('test-image-person-group.jpg')
image = open(test_image_array[0], 'r+b')

# print("Pausing for 60 seconds to avoid triggering rate limit on free account...")
# time.sleep(60)

# Detect faces
face_ids =[]
faces = face_client.face.detect_with_stream(image, detectionModel='detection_02')
for face in faces:
    face_ids.append(face.face_id)

In [38]:
# Identify faces
# takes array of detected faces and compares to the person group

results = face_client.face.identify(face_ids, PERSON_GROUP_ID)
print('Identify faces in {}'.format(os.path.basename(image.name)))
if not results:
    print('No person identified in the person group for faces from {}'.format(os.path.basename(image.name)))
for person in results:
    if len(person.candidates) >0:
        print('Person for face ID {} is identified in {} with a confidence of {}.'.format(person.face_id, os.path.basename(image.name), person.candidates[0].confidence))
    else:
        print('No person identified for face ID {} in {}.'.format(person.face_id, os.path.basename(image.name)))

Identify faces in test-image-person-group.jpg
Person for face ID 8261b695-3b4f-4842-9eef-48e54a5f755e is identified in test-image-person-group.jpg with a confidence of 0.92387.
Person for face ID 2d8ae1d8-3834-4e33-9549-a76c7dc72f41 is identified in test-image-person-group.jpg with a confidence of 0.93316.


## Verify Faces

verifies face_ids with other face_id or person object and determines if they belong to same person


**Getting Test Images**

In [39]:

IMAGE_BASE_URL = 'https://csdx.blob.core.windows.net/resources/Face/Images/'

# create a list to hold the target photos of the same person
target_image_file_names = ['Family1-Dad1.jpg', 'Family1-Dad2.jpg']
# The source photos contain this person
source_image_file_name1 = 'Family1-Dad3.jpg'
source_image_file_name2 = 'Family1-Son1.jpg'

**Detect faces for verification**

In [42]:
# Detect faces from source image 1, returns list[DetectedFaces]
detected_faces1 = face_client.face.detect_with_url(IMAGE_BASE_URL + source_image_file_name1, detectionModel = 'detection_02')
#Add the returned face's face id
source_image1_id = detected_faces1[0].face_id
print('{} face(s) detected from image {}.'.format(len(detected_faces1), source_image_file_name1))

# Detect faces from source image 2, returns list[DetectedFaces]
detected_faces2 = face_client.face.detect_with_url(IMAGE_BASE_URL + source_image_file_name2, detcetionModel='detection_02')
source_image2_id = detected_faces2[0].face_id
print('{} faces(s) detected from image {}.'.format(len(detected_faces2),source_image_file_name2))

# List for the target face IDs (uuids)
detected_faces_ids=[]
# Detect faces from the target image url list, returns a list[Detect#edFaces]
for image_file_name in target_image_file_names:
    detected_faces = face_client.face.detect_with_url(IMAGE_BASE_URL + image_file_name, detectionModel ='detection_02')
    detected_faces_ids.append(detected_faces[0].face_id)
    print('{} face(s) detected from image {}.'.format(len(detected_faces), image_file_name))

1 face(s) detected from image Family1-Dad3.jpg.
1 faces(s) detected from image Family1-Son1.jpg.
1 face(s) detected from image Family1-Dad1.jpg.
1 face(s) detected from image Family1-Dad2.jpg.


**Get verification Results**

In [43]:
# comparing each source image to the target image and print a messae if they belong to same person
# Since target faces are the same person, in this example, we can use the 1st ID in the detected_faces_ids list to compare.
verify_result_same = face_client.face.verify_face_to_face(source_image1_id, detected_faces_ids[0])
print('Faces from {} & {} are of the same person, with confidence: {}'
    .format(source_image_file_name1, target_image_file_names[0], verify_result_same.confidence)
    if verify_result_same.is_identical
    else 'Faces from {} & {} are of a different person, with confidence: {}'
        .format(source_image_file_name1, target_image_file_names[0], verify_result_same.confidence))

# Verification example for faces of different persons
verify_result_diff = face_client.face.verify_face_to_face(source_image2_id, detected_faces_ids[0])
print('Faces from {} & {} are of the same person, with confidence: {}'
    .format(source_image_file_name2, target_image_file_names[0], verify_result_diff.confidence)
    if verify_result_diff.is_identical
    else 'Faces from {} & {} are of a different person, with confidence: {}'
        .format(source_image_file_name2, target_image_file_names[0], verify_result_diff.confidence))

Faces from Family1-Dad3.jpg & Family1-Dad1.jpg are of the same person, with confidence: 0.88193
Faces from Family1-Son1.jpg & Family1-Dad1.jpg are of a different person, with confidence: 0.26628


## Deleting the main Person Group

In [44]:
face_client.person_group.delete(person_group_id=PERSON_GROUP_ID)
print("Deleted the person group {} from the source location.".format(PERSON_GROUP_ID))
print()

Deleted the person group e6fcc27f-7910-4fe5-857c-b6ccbbf0cf44 from the source location.

